In [1]:
import cv2
import matplotlib.pyplot as plt
import mediapipe as mp
import numpy as np
import os 
import tensorflow.keras.models

In [2]:
# mediapipe propagation
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_holistic_bb = mp.solutions.holistic.Holistic()

In [3]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results = model.process(image)
    image.flags.writeable=True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results
def draw_img(image):
    results = mp_holistic_bb.process(image)
    hand_landmarks = results.right_hand_landmarks or results.left_hand_landmarks
    if hand_landmarks is not None:
        xmin, ymin, xmax, ymax = float('inf'), float('inf'), 0, 0
        for landmark in hand_landmarks.landmark:
            x, y = int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0])
            xmin, ymin = min(xmin, x), min(ymin, y)
            xmax, ymax = max(xmax, x), max(ymax, y)
        cv2.rectangle(image, (xmin-10, ymin-10), (xmax+10, ymax+10), (0, 255, 0), 2)
    return image
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks , mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks , mp_holistic.HAND_CONNECTIONS) # hand drawing

In [4]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence= 0.5,min_tracking_confidence= 0.5) as holistic:
  while True:
    ret,frame = cap.read()
    frame,results = mediapipe_detection(frame,holistic)
    frame = draw_img(frame)
    draw_landmarks(frame,results)
    cv2.imshow('cam feed',frame)
    key = cv2.waitKey(1)
    if ord('q')==key:
      break
  cap.release()
  cv2.destroyAllWindows()

In [5]:
def extract_keypoints(results):
#     face = np.array([[marks.x,marks.y,marks.z] for marks in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    pose = np.array([[marks.x,marks.y,marks.z,marks.visibility] for marks in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    lh = np.array([[marks.x,marks.y,marks.z] for marks in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[marks.x,marks.y,marks.z] for marks in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
#     return np.concatenate([pose,face,lh,rh])
    return np.concatenate([pose,lh,rh])

In [6]:
path = os.path.join(r"C:\Users\venka\Desktop\personal\Computer_vision\Action_detection")
actions = np.array(['hello','stand','walking','sit','jump'])
no_sequences = 30
sequence_length = 30

In [7]:
for i in actions:
    for s in range(no_sequences):
        try:
            os.makedirs(os.path.join(path,i,str(s)))
        except:
            pass

In [8]:
cap = cv2.VideoCapture(1)

with mp_holistic.Holistic(min_detection_confidence= 0.5,min_tracking_confidence= 0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                
                ret,frame = cap.read()

                frame,results = mediapipe_detection(frame,holistic)
                draw_landmarks(frame,results)
                if frame_num==0:
                    cv2.putText(frame,'start',(120,200),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.putText(frame,f'collecting frames for {action} video Number {sequence}',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.imshow('img feed',frame)
                    cv2.waitKey(200)
                else:
                    cv2.putText(frame,f'collecting frames for {action} video Number {sequence}',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.imshow('img feed',frame)
                keypoints = extract_keypoints(results)
                n_path = os.path.join(path,action,str(sequence),str(frame_num))
                np.save(n_path,keypoints)
                
                key = cv2.waitKey(1)
                if ord('q')==key:
                    break
    cap.release()
    cv2.destroyAllWindows() 

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num , label in enumerate(actions)}

In [ ]:
sequences , labels = [],[]
for action in actions:
    for sequence in range(no_sequences):
        window=[]
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(path,action,str(sequence),f"{frame_num}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.05,random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPool1D
from tensorflow.keras.callbacks import TensorBoard

ImportError: cannot import name 'Con' from 'tensorflow.keras.layers' (c:\Users\venka\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\api\_v2\keras\layers\__init__.py)

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(Conv1D(64,1,input_shape=(543,3),activation="relu"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 543, 64)           256       
                                                                 
Total params: 256
Trainable params: 256
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True,activation = 'relu',input_shape=(30,258)))
model.add(LSTM(128, return_sequences=True,activation = 'relu'))
model.add(LSTM(64, return_sequences=False,activation = 'relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='Softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
model.fit(x_train,y_train,epochs=700,callbacks = [tb_callback])

Epoch 1/700
5/5 [==============================] - 3s 35ms/step - loss: 1.6032 - categorical_accuracy: 0.1831
Epoch 2/700
5/5 [==============================] - 0s 32ms/step - loss: 1.4235 - categorical_accuracy: 0.4296
Epoch 3/700
5/5 [==============================] - 0s 33ms/step - loss: 1.5734 - categorical_accuracy: 0.3451
Epoch 4/700
5/5 [==============================] - 0s 33ms/step - loss: 1.4419 - categorical_accuracy: 0.4789
Epoch 5/700
5/5 [==============================] - 0s 32ms/step - loss: 1.6565 - categorical_accuracy: 0.5915
Epoch 6/700
5/5 [==============================] - 0s 31ms/step - loss: 1.3795 - categorical_accuracy: 0.4296
Epoch 7/700
5/5 [==============================] - 0s 32ms/step - loss: 1.1976 - categorical_accuracy: 0.6620
Epoch 8/700
5/5 [==============================] - 0s 32ms/step - loss: 1.2592 - categorical_accuracy: 0.6901
Epoch 9/700
5/5 [==============================] - 0s 36ms/step - loss: 1.3183 - categorical_accuracy: 0.4859
Epoch 10/7

5/5 [==============================] - 0s 30ms/step - loss: 0.6326 - categorical_accuracy: 0.8451
Epoch 75/700
5/5 [==============================] - 0s 31ms/step - loss: 0.5308 - categorical_accuracy: 0.8592
Epoch 76/700
5/5 [==============================] - 0s 31ms/step - loss: 0.5294 - categorical_accuracy: 0.8380
Epoch 77/700
5/5 [==============================] - 0s 32ms/step - loss: 0.5817 - categorical_accuracy: 0.7887
Epoch 78/700
5/5 [==============================] - 0s 36ms/step - loss: 0.5552 - categorical_accuracy: 0.7746
Epoch 79/700
5/5 [==============================] - 0s 34ms/step - loss: 0.3896 - categorical_accuracy: 0.8732
Epoch 80/700
5/5 [==============================] - 0s 32ms/step - loss: 0.4817 - categorical_accuracy: 0.8944
Epoch 81/700
5/5 [==============================] - 0s 31ms/step - loss: 2.0641 - categorical_accuracy: 0.7606
Epoch 82/700
5/5 [==============================] - 0s 31ms/step - loss: 0.4980 - categorical_accuracy: 0.8521
Epoch 83/700
5

5/5 [==============================] - 0s 32ms/step - loss: 0.2226 - categorical_accuracy: 0.9366
Epoch 148/700
5/5 [==============================] - 0s 30ms/step - loss: 0.2167 - categorical_accuracy: 0.9366
Epoch 149/700
5/5 [==============================] - 0s 30ms/step - loss: 0.2183 - categorical_accuracy: 0.9296
Epoch 150/700
5/5 [==============================] - 0s 31ms/step - loss: 0.1604 - categorical_accuracy: 0.9437
Epoch 151/700
5/5 [==============================] - 0s 34ms/step - loss: 0.1394 - categorical_accuracy: 0.9648
Epoch 152/700
5/5 [==============================] - 0s 33ms/step - loss: 0.1302 - categorical_accuracy: 0.9507
Epoch 153/700
5/5 [==============================] - 0s 33ms/step - loss: 0.1170 - categorical_accuracy: 0.9507
Epoch 154/700
5/5 [==============================] - 0s 32ms/step - loss: 0.1108 - categorical_accuracy: 0.9577
Epoch 155/700
5/5 [==============================] - 0s 31ms/step - loss: 0.1016 - categorical_accuracy: 0.9718
Epoch 

5/5 [==============================] - 0s 44ms/step - loss: 0.5744 - categorical_accuracy: 0.7535
Epoch 221/700
5/5 [==============================] - 0s 44ms/step - loss: 0.5144 - categorical_accuracy: 0.7817
Epoch 222/700
5/5 [==============================] - 0s 48ms/step - loss: 0.4892 - categorical_accuracy: 0.7958
Epoch 223/700
5/5 [==============================] - 0s 45ms/step - loss: 0.4669 - categorical_accuracy: 0.7817
Epoch 224/700
5/5 [==============================] - 0s 47ms/step - loss: 0.4259 - categorical_accuracy: 0.8169
Epoch 225/700
5/5 [==============================] - 0s 48ms/step - loss: 0.4139 - categorical_accuracy: 0.7958
Epoch 226/700
5/5 [==============================] - 0s 42ms/step - loss: 0.3840 - categorical_accuracy: 0.8099
Epoch 227/700
5/5 [==============================] - 0s 41ms/step - loss: 0.3756 - categorical_accuracy: 0.8239
Epoch 228/700
5/5 [==============================] - 0s 43ms/step - loss: 0.3470 - categorical_accuracy: 0.8380
Epoch 

5/5 [==============================] - 0s 42ms/step - loss: 0.1753 - categorical_accuracy: 0.9366
Epoch 294/700
5/5 [==============================] - 0s 43ms/step - loss: 0.1673 - categorical_accuracy: 0.9366
Epoch 295/700
5/5 [==============================] - 0s 38ms/step - loss: 0.1543 - categorical_accuracy: 0.9507
Epoch 296/700
5/5 [==============================] - 0s 40ms/step - loss: 0.1531 - categorical_accuracy: 0.9437
Epoch 297/700
5/5 [==============================] - 0s 40ms/step - loss: 0.1502 - categorical_accuracy: 0.9648
Epoch 298/700
5/5 [==============================] - 0s 37ms/step - loss: 0.1468 - categorical_accuracy: 0.9437
Epoch 299/700
5/5 [==============================] - 0s 47ms/step - loss: 0.1449 - categorical_accuracy: 0.9437
Epoch 300/700
5/5 [==============================] - 0s 54ms/step - loss: 0.1223 - categorical_accuracy: 0.9437
Epoch 301/700
5/5 [==============================] - 0s 51ms/step - loss: 0.1206 - categorical_accuracy: 0.9507
Epoch 

5/5 [==============================] - 0s 42ms/step - loss: 0.1638 - categorical_accuracy: 0.9577
Epoch 367/700
5/5 [==============================] - 0s 64ms/step - loss: 0.1513 - categorical_accuracy: 0.9507
Epoch 368/700
5/5 [==============================] - 0s 48ms/step - loss: 0.1468 - categorical_accuracy: 0.9437
Epoch 369/700
5/5 [==============================] - 0s 50ms/step - loss: 0.1389 - categorical_accuracy: 0.9577
Epoch 370/700
5/5 [==============================] - 0s 56ms/step - loss: 0.1384 - categorical_accuracy: 0.9507
Epoch 371/700
5/5 [==============================] - 0s 51ms/step - loss: 0.1341 - categorical_accuracy: 0.9577
Epoch 372/700
5/5 [==============================] - 0s 51ms/step - loss: 0.1299 - categorical_accuracy: 0.9577
Epoch 373/700
5/5 [==============================] - 0s 54ms/step - loss: 0.1261 - categorical_accuracy: 0.9648
Epoch 374/700
5/5 [==============================] - 0s 55ms/step - loss: 0.1214 - categorical_accuracy: 0.9648
Epoch 

5/5 [==============================] - 0s 47ms/step - loss: 0.0439 - categorical_accuracy: 0.9859
Epoch 440/700
5/5 [==============================] - 0s 49ms/step - loss: 0.0410 - categorical_accuracy: 0.9859
Epoch 441/700
5/5 [==============================] - 0s 47ms/step - loss: 0.1783 - categorical_accuracy: 0.9718
Epoch 442/700
5/5 [==============================] - 0s 46ms/step - loss: 0.2857 - categorical_accuracy: 0.9437
Epoch 443/700
5/5 [==============================] - 0s 53ms/step - loss: 0.2114 - categorical_accuracy: 0.9366
Epoch 444/700
5/5 [==============================] - 0s 56ms/step - loss: 0.2318 - categorical_accuracy: 0.9225
Epoch 445/700
5/5 [==============================] - 0s 56ms/step - loss: 0.1375 - categorical_accuracy: 0.9577
Epoch 446/700
5/5 [==============================] - 0s 49ms/step - loss: 0.1184 - categorical_accuracy: 0.9718
Epoch 447/700
5/5 [==============================] - 0s 50ms/step - loss: 0.1088 - categorical_accuracy: 0.9648
Epoch 

5/5 [==============================] - 0s 57ms/step - loss: 1.0816 - categorical_accuracy: 0.6690
Epoch 513/700
5/5 [==============================] - 0s 51ms/step - loss: 0.7869 - categorical_accuracy: 0.6761
Epoch 514/700
5/5 [==============================] - 0s 55ms/step - loss: 0.6824 - categorical_accuracy: 0.6901
Epoch 515/700
5/5 [==============================] - 0s 48ms/step - loss: 0.5852 - categorical_accuracy: 0.7887
Epoch 516/700
5/5 [==============================] - 0s 54ms/step - loss: 0.5287 - categorical_accuracy: 0.8521
Epoch 517/700
5/5 [==============================] - 0s 62ms/step - loss: 0.4731 - categorical_accuracy: 0.8662
Epoch 518/700
5/5 [==============================] - 0s 56ms/step - loss: 0.4334 - categorical_accuracy: 0.8592
Epoch 519/700
5/5 [==============================] - 0s 60ms/step - loss: 0.3962 - categorical_accuracy: 0.8873
Epoch 520/700
5/5 [==============================] - 0s 59ms/step - loss: 0.3549 - categorical_accuracy: 0.8521
Epoch 

5/5 [==============================] - 0s 56ms/step - loss: 0.5002 - categorical_accuracy: 0.7606
Epoch 586/700
5/5 [==============================] - 0s 56ms/step - loss: 0.4339 - categorical_accuracy: 0.8380
Epoch 587/700
5/5 [==============================] - 0s 58ms/step - loss: 0.4633 - categorical_accuracy: 0.8169
Epoch 588/700
5/5 [==============================] - 0s 58ms/step - loss: 0.4349 - categorical_accuracy: 0.8310
Epoch 589/700
5/5 [==============================] - 0s 69ms/step - loss: 0.4939 - categorical_accuracy: 0.8028
Epoch 590/700
5/5 [==============================] - 0s 63ms/step - loss: 0.4444 - categorical_accuracy: 0.8380
Epoch 591/700
5/5 [==============================] - 0s 55ms/step - loss: 0.4224 - categorical_accuracy: 0.8310
Epoch 592/700
5/5 [==============================] - 0s 34ms/step - loss: 0.4125 - categorical_accuracy: 0.8451
Epoch 593/700
5/5 [==============================] - 0s 46ms/step - loss: 0.3993 - categorical_accuracy: 0.8662
Epoch 

5/5 [==============================] - 0s 43ms/step - loss: 0.2593 - categorical_accuracy: 0.9014
Epoch 659/700
5/5 [==============================] - 0s 52ms/step - loss: 0.3303 - categorical_accuracy: 0.8803
Epoch 660/700
5/5 [==============================] - 0s 62ms/step - loss: 0.3455 - categorical_accuracy: 0.8944
Epoch 661/700
5/5 [==============================] - 0s 66ms/step - loss: 0.2162 - categorical_accuracy: 0.9366
Epoch 662/700
5/5 [==============================] - 0s 64ms/step - loss: 0.2067 - categorical_accuracy: 0.9437
Epoch 663/700
5/5 [==============================] - 0s 60ms/step - loss: 0.1629 - categorical_accuracy: 0.9437
Epoch 664/700
5/5 [==============================] - 0s 59ms/step - loss: 0.1493 - categorical_accuracy: 0.9648
Epoch 665/700
5/5 [==============================] - 0s 66ms/step - loss: 0.1315 - categorical_accuracy: 0.9437
Epoch 666/700
5/5 [==============================] - 0s 56ms/step - loss: 0.1273 - categorical_accuracy: 0.9577
Epoch 

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_14 (LSTM)              (None, 30, 64)            82688     
                                                                 
 lstm_15 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_16 (LSTM)              (None, 64)                49408     
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 5)                 165       
                                                                 
Total params: 237,317
Trainable params: 237,317
Non-tr

In [ ]:
res = model.predict(x_test)

1/1 [==============================] - 0s 489ms/step


In [ ]:
model.evaluate(x,y)

5/5 [==============================] - 1s 20ms/step - loss: 0.0451 - categorical_accuracy: 0.9933


[0.045084722340106964, 0.9933333396911621]

In [ ]:
model.save(r"C:\Users\venka\Desktop\personal\Deep_learning\My_lstm.h5")

In [ ]:
model=tensorflow.keras.models.load_model(r"C:\Users\venka\Desktop\personal\Deep_learning\My_lstm.h5")

In [ ]:
from scipy import stats

In [ ]:
fc = cv2.CascadeClassifier(r"C:\Users\venka\Desktop\personal\Computer_vision\cascade_classifiers\harcascadeclassifier.xml")

In [ ]:
def imag(a,image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = fc.detectMultiScale(
        gray,
        scaleFactor=1.5,
        minNeighbors=5,
        minSize=(30, 30),
        )
# print("Found {0} faces!".format(len(faces)))
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.putText(image, actions[a], (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    return image

In [ ]:
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
a=0
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            a=np.argmax(res)
            cv2.putText(image, actions[a], (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 19ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
hello
<class 'mediapipe.pyth

1/1 [==============================] - 0s 21ms/step
jump
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
sit
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
sit
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
jump
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
jump
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
jump
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
jump
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
sit
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
sit
<class 'mediapipe.python.solution_b

1/1 [==============================] - 0s 21ms/step
sit
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
sit
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
sit
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
sit
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
sit
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
sit
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
sit
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
sit
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
sit
<class 'mediapipe.python.solution_base.S

hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [============

1/1 [==============================] - 0s 22ms/step
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
hello


In [ ]:
actions[a]

'hello'